# Step 2. 전문가의 말을 쉽게 풀어 재구성된 text 생성
1. UpStage의 자연어처리api, Solar LLM 사용.
   : Solar LLM api를 사용하여 텍스트를 분석하고, 더 쉬운 단어와 문장으로 재구성
2. 문서 검색 기반 RAG모델 구축
   : (1) 사전에 구축한 의료 문서 DB or (2) 헬스케어 분야를 특정하는 프롬프트를 이용한 웹크롤링을 통해 관련 정보 검색 후 --> 이를 텍스트로 통합해 재구성; embeding도 upstage embeding 사용할 예정이라 A-1 구조로 설계할 예정

* 1단계. Solar LLM model을 활용해 원본 텍스트를 분석하고, 이를 더 쉬운 단어와 문장으로 유연하게 재구성

In [2]:
#!pip install reportlab

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB 163.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/1.9 MB 178.6 kB/s eta 0:00:11
   - -------------------------------------- 0.1/1.9 MB 281.8 kB/s eta 0:00:07
   ---- ----------------------------------- 0.2/1.9 MB 695.5 kB/s eta 0:00:03
   ------- -------------------------------- 0.4/1.9 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/1.9 MB 1.3 MB/s eta 0:00:02
   ----------- ---------------------------- 0.6/1.9 MB 1.4 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/1.9 MB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/1.9 MB 1.7 MB/s eta 0:00:01
   ------------------- -----

DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
import json
import os
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_upstage import ChatUpstage
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Load environment variables from .env
load_dotenv()

def read_medical_report(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data.get('transcription', '')

def simplify_medical_report(llm, medical_report):
    template = """
    아래는 환자 진료 중 의사의 소견입니다. 이 내용을 비전문가인 일반인도 쉽게 이해할 수 있도록 간단하고 명확한 언어로 다시 설명해주세요. 다음의 지침을 따라주세요.

    0. 의사의 발음은 정확하지 않습니다. 따라서 발음상 text로 잘못 옮겨진 부분이 있을텐데, 우선 문맥을 파악하여 그러한 부분을 수정해주세요.
    1. 의학 용어는 가능한 쉬운 말로 바꾸고, 필요시 간단한 설명을 덧붙이세요. 
    2. 전문적인 내용은 일상생활의 비유나 예시를 들어 설명해주세요. 
    3. 환자의 상태와 치료 계획을 가능한 경우 단계별로 설명해주세요. 
    4. 중요한 의학 정보는 반드시 포함하고, 누락되지 않도록 주의해주세요. 
    5. 환자가 취해야 할 행동이나 주의사항을 명확히 강조해주세요. 
    6. 전체 내용을 3-5개의 주요 섹션으로 나누어 구조화해주세요. 

    의사 소견서:
    {medical_report}
    """
    
    prompt_template = ChatPromptTemplate.from_template(template)
    
    prompt = prompt_template.format_messages(medical_report=medical_report)
    
    response = llm.invoke(prompt)
    
    return response

# 생성된 텍스트를 TXT 파일 형태로 저장
def save_to_txt(text, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

#생성된 텍스트를 PDF 파일 형태로 저장
def save_to_pdf(text, file_path):
    #폰트 파일을 직접 로드
    pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothic.ttf'))
    c = canvas.Canvas(file_path, pagesize=letter)
    c.setFont('NanumGothic', 12)

    #텍스트 쓰기
    text_object = c.beginText(50, 750)
    text_object.setFont('NanumGothic', 12)

    for line in text.split('\n'):
        text_object.textLine(line)

    c.drawText(text_object)
    c.save()    

def main():
    # API 키를 환경 변수에서 가져옵니다
    api_key = os.getenv("UPSTAGE_API_KEY")
    if not api_key:
        raise ValueError("UPSTAGE_API_KEY not found in .env file")

    # Solar LLM 모델 초기화
    llm = ChatUpstage(api_key=api_key)

    file_path = "transcription.json"

    medical_report = read_medical_report(file_path)
    simplified_report = simplify_medical_report(llm, medical_report)

    #생성된 텍스트를 파일에 저장
    save_to_pdf(simplified_report.content, "simplified_medical_report.pdf")
    save_to_txt(simplified_report.content, "simplified_medical_report.txt")
    
    print("간소화된 의사 소견:")
    print(simplified_report.content)

if __name__ == "__main__":
    main()

간소화된 의사 소견:
I. 소개

* 환자의 보험은 2차 소견을 구하는 특정한 보험을 사용합니다.

II. 2차 소견 요청

* 환자는 본인의 주치의에게 다른 의사가 전문의의 추천을 요청할 수 있습니다.
* 의사들은 대부분 다른 의견을 환영합니다.
* 두 번째 의사는 첫 번째 의사와 동일한 의견을 가지지 않을 수 있습니다.

III. 두 번째 의사 선택

* 환자는 첫 번째 의사와 동일한 의견을 가지지 않는 두 번째 의사를 선임할 수 있습니다.
* 두 번째 의사는 대학병원, 전문의료협회, 또는 보험회사에서 선임할 수 있습니다.

IV. 환자 행동 및 주의사항

* 환자는 본인이 신뢰하는 다른 의사에게 2차 소견을 요청할 수 있습니다.
* 두 번째 의사를 선임할 수 없는 경우, 대학병원, 전문의료협회, 또는 보험회사에서 선임할 수 있습니다.

V. 치료 계획

* 구체적인 치료 계획은 환자의 상태와 의사의 판단에 따라 달라집니다.

중요한 의학 정보:

* 환자의 보험은 2차 소견을 구하는 특정한 보험을 사용합니다.
* 환자는 주치의에게 다른 의사가 전문의의 추천을 요청할 수 있습니다.
* 두 번째 의사는 첫 번째 의사와 동일한 의견을 가지지 않을 수 있습니다.
* 환자는 두 번째 의사를 선임할 수 있습니다.

환자가 취해야 할 행동:

* 주치의에게 2차 소견을 요청할 수 있습니다.
* 두 번째 의사를 선임할 수 없는 경우, 대학병원, 전문의료협회, 또는 보험회사에서 선임할 수 있습니다.

환자가 주의해야 할 사항:

* 두 번째 의사는 첫 번째 의사와 동일한 의견을 가지지 않을 수 있습니다.
* 두 번째 의사를 선임할 수 없는 경우, 다른 방법을 찾아야 합니다.


In [7]:
#Step 5에서 context(입력값)에 대한 토큰수 제한(<32000)이 있어서 글자수에 제한을 두고 요약본을 추가 생성

import os
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_upstage import ChatUpstage
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Load environment variables from .env
load_dotenv()

def read_simplified_report(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def summarize_report(llm, simplified_report):
    template = """
    다음은 간소화된 의사 소견입니다. 이 내용을 32000자 이내로 요약해주세요. 
    중요한 의학 정보와 환자가 취해야 할 행동이나 주의사항을 누락하지 않도록 주의해주세요.

    간소화된 의사 소견:
    {simplified_report}
    """
    
    prompt_template = ChatPromptTemplate.from_template(template)
    prompt = prompt_template.format_messages(simplified_report=simplified_report)
    
    response = llm.invoke(prompt)
    return response.content

def save_to_pdf(text, file_path):
    pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothic.ttf'))
    c = canvas.Canvas(file_path, pagesize=letter)
    c.setFont('NanumGothic', 12)
    text_object = c.beginText(50, 750)
    text_object.setFont('NanumGothic', 12)
    for line in text.split('\n'):
        text_object.textLine(line)
    c.drawText(text_object)
    c.save()

def main():
    # API 키를 환경 변수에서 가져옵니다
    api_key = os.getenv("UPSTAGE_API_KEY")
    if not api_key:
        raise ValueError("UPSTAGE_API_KEY not found in .env file")

    # Solar LLM 모델 초기화
    llm = ChatUpstage(api_key=api_key)

    # 간소화된 의사 소견 읽기
    simplified_report = read_simplified_report("simplified_medical_report.txt")

    # 간소화된 보고서 요약
    summarized_report = summarize_report(llm, simplified_report)

    # 요약된 보고서를 PDF 파일로 저장
    save_to_pdf(summarized_report, "simplified_medical_report_summary.pdf")

    print("요약된 의사 소견이 'simplified_medical_report_summary.pdf'로 저장되었습니다.")

if __name__ == "__main__":
    main()

요약된 의사 소견이 'simplified_medical_report_summary.pdf'로 저장되었습니다.
